In [11]:
import pandas as pd
from models.common.util import convert_utc_to_kst, set_weather_key

In [15]:
weather = pd.read_csv("../data/weather_rapid.csv")

In [16]:
weather.head(5)

,select_timestamp,select_date,announce_timestamp,announceF_timestamp,kma_announce_timestamp,code,areaname_wide,areaname_city,areaname_dong,wtext,...,wspd_3hr,wspd_4hr,wspd_5hr,wspd_6hr,pty_1hr,pty_2hr,pty_3hr,pty_4hr,pty_5hr,pty_6hr
0,2023-11-21 02:20:00 UTC,2023-11-21,2023-11-21 01:00:00 UTC,2023-11-21 02:00:00 UTC,2023-11-21 01:30:00 UTC,1,서울특별시,-,-,맑음,...,1,1,1,1,0,0,0,0,0,0
1,2023-11-21 01:40:00 UTC,2023-11-21,2023-11-21 01:00:00 UTC,2023-11-21 02:00:00 UTC,2023-11-21 01:30:00 UTC,1,서울특별시,-,-,맑음,...,1,1,1,1,0,0,0,0,0,0
2,2023-11-21 01:50:00 UTC,2023-11-21,2023-11-21 01:00:00 UTC,2023-11-21 02:00:00 UTC,2023-11-21 01:30:00 UTC,1,서울특별시,-,-,맑음,...,1,1,1,1,0,0,0,0,0,0
3,2023-11-21 02:10:00 UTC,2023-11-21,2023-11-21 01:00:00 UTC,2023-11-21 02:00:00 UTC,2023-11-21 01:30:00 UTC,1,서울특별시,-,-,맑음,...,1,1,1,1,0,0,0,0,0,0
4,2023-11-21 02:00:00 UTC,2023-11-21,2023-11-21 01:00:00 UTC,2023-11-21 02:00:00 UTC,2023-11-21 01:30:00 UTC,1,서울특별시,-,-,맑음,...,1,1,1,1,0,0,0,0,0,0


In [12]:
weather = pd.read_csv("../data/weather_rapid.csv")
weather = weather[["select_timestamp", "areaname_wide", "areaname_city", "wtext", "wtext_1hr", "rain", "snow", "wtext_1hr", "rain_1hr", "temp", "temp_1hr"]]

weather["snow"] = weather["snow"].fillna(0)
weather["select_timestamp"] = weather["select_timestamp"].apply(convert_utc_to_kst)

In [13]:
weather["key"] = weather[["select_timestamp", "areaname_wide", "areaname_city"]].apply(
    lambda x: set_weather_key(
        select_timestamp=x.get("select_timestamp"),
        wide=x.get("areaname_wide"),
        city=x.get("areaname_city")
    ), axis=1
)

In [14]:
weather = weather.drop_duplicates(subset='key', keep='first')
weather.set_index("key").to_dict(orient="index")

/var/folders/_k/2zc2tyfs1dld9dgj9c9mt9gm0000gn/T/ipykernel_59896/3440300387.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  weather.set_index("key").to_dict(orient="index")


{'2023-11-21T11:00__서울특별시': {'select_timestamp': '2023-11-21T11:00',
  'areaname_wide': '서울특별시',
  'areaname_city': '-',
  'wtext': '맑음',
  'wtext_1hr': '맑음',
  'rain': 0,
  'snow': 0.0,
  'rain_1hr': 0},
 '2023-11-21T10:00__서울특별시': {'select_timestamp': '2023-11-21T10:00',
  'areaname_wide': '서울특별시',
  'areaname_city': '-',
  'wtext': '맑음',
  'wtext_1hr': '맑음',
  'rain': 0,
  'snow': 0.0,
  'rain_1hr': 0},
 '2023-11-21T11:00__서울특별시__종로구': {'select_timestamp': '2023-11-21T11:00',
  'areaname_wide': '서울특별시',
  'areaname_city': '종로구',
  'wtext': '맑음',
  'wtext_1hr': '맑음',
  'rain': 0,
  'snow': 0.0,
  'rain_1hr': 0},
 '2023-11-21T10:00__서울특별시__종로구': {'select_timestamp': '2023-11-21T10:00',
  'areaname_wide': '서울특별시',
  'areaname_city': '종로구',
  'wtext': '맑음',
  'wtext_1hr': '맑음',
  'rain': 0,
  'snow': 0.0,
  'rain_1hr': 0},
 '2023-11-21T11:00__서울특별시__중구': {'select_timestamp': '2023-11-21T11:00',
  'areaname_wide': '서울특별시',
  'areaname_city': '중구',
  'wtext': '맑음',
  'wtext_1hr': '맑음',
  

In [5]:
weather = weather.rename(
    columns={
        "select_timestamp":"현재시간",
        "areaname_wide":"지역명",
        "areaname_city": "도시명",
        "wtext" : "현재날씨",
        "wtext_1hr" : "1시간 뒤 예측 날씨",
        "rain" : "강수량(cm)",
        "snow" : "강설량(cm)",
        "rain_1hr" : "1시간 뒤 예측 강수량(cm)"
    }
)

In [6]:
weather = weather.drop_duplicates(subset='key', keep='first')
weather_db = weather.set_index("key").to_dict(orient="index")

/var/folders/_k/2zc2tyfs1dld9dgj9c9mt9gm0000gn/T/ipykernel_59896/505943760.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  weather_db = weather.set_index("key").to_dict(orient="index")


In [7]:
data = []
for key, value in weather_db["2023-11-21 02:10__서울특별시__도봉구"].items():
    data.append(f"{key} : {value}")

KeyError: '2023-11-21 02:10__서울특별시__도봉구'

In [ ]:
data

In [8]:
from models.common.util import get_weather_db

db = get_weather_db("../data/weather_rapid.csv")

/Users/moon.dev/Desktop/airnd/ai-hackerton/models/common/util.py:81: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return weather.set_index("key").to_dict(orient="index")


In [9]:
from models.common.util import get_current_weather_info

a = get_current_weather_info(db, areaname_wide="서울특별시", areaname_city="강남구")

In [10]:
from models.common.util import get_current_kst_time

current_time = get_current_kst_time()
set_weather_key(current_time, "서울특별시", "강남구")

'2023-11-22T20:00__서울특별시__강남구'